# 03 - Porte e Benchmark de Distribuidoras

Comparacao mensal e anual entre distribuidoras com normalizacao por UC ativa.

In [ ]:
from pathlib import Path
import pandas as pd

ROOT = Path.cwd()
if not (ROOT / 'data').exists():
    ROOT = ROOT.parent
ANALYSIS_DIR = ROOT / 'data' / 'processed' / 'analysis'

dist_mes = pd.read_parquet(ANALYSIS_DIR / 'fato_transgressao_mensal_distribuidora.parquet')
porte = pd.read_parquet(ANALYSIS_DIR / 'dim_distribuidora_porte.parquet')

In [ ]:
# Visao mensal consolidada por distribuidora (ja pronta)
dist_mes.head()

In [ ]:
# Benchmark focado: Coelba, Neoenergia Brasilia, Copel
focus_terms = ['ESTADO DA BAHIA', 'BRASILIA', 'COPEL']
focus_names = []
for term in focus_terms:
    matches = [n for n in dist_mes['nomagente'].dropna().unique() if term in n.upper()]
    if matches:
        focus_names.append(matches[0])

bench = dist_mes[dist_mes['nomagente'].isin(focus_names)].copy()
bench.sort_values(['ano', 'mes', 'nomagente']).head(40)

In [ ]:
# Ranking por taxa normalizada no periodo mais recente
ultimo_ano = dist_mes['ano'].max()
rank = (
    dist_mes[dist_mes['ano'] == ultimo_ano]
    .groupby(['sigagente', 'nomagente'], as_index=False)
    .agg(
        qtd_serv_realizado=('qtd_serv_realizado', 'sum'),
        qtd_fora_prazo=('qtd_fora_prazo', 'sum'),
        compensacao_rs=('compensacao_rs', 'sum'),
        fora_prazo_por_100k_uc_mes=('fora_prazo_por_100k_uc_mes', 'mean'),
    )
)
rank['taxa_fora_prazo'] = rank['qtd_fora_prazo'] / rank['qtd_serv_realizado']
rank.sort_values('fora_prazo_por_100k_uc_mes', ascending=False).head(30)